In [1]:
import pandas as pd
import numpy as np
import cPickle as pickle

In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
prop = pickle.load(open("Data.p","rb"))
prop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72474 entries, 0 to 72473
Data columns (total 35 columns):
id                            72474 non-null object
created_on                    72474 non-null object
operation                     72474 non-null object
property_type                 72474 non-null object
place_name                    72446 non-null object
place_with_parent_names       72474 non-null object
country_name                  72474 non-null object
state_name                    72474 non-null object
geonames_id                   62030 non-null float64
lat-lon                       48174 non-null object
lat                           48174 non-null float64
lon                           48174 non-null float64
price                         72474 non-null float64
currency                      72473 non-null object
price_aprox_local_currency    72474 non-null float64
price_aprox_usd               72474 non-null float64
surface_total_in_m2           72474 non-null float64


In [4]:
len(prop)

72474

In [5]:
#cortamos por precios
prop = prop[(prop['price_usd_per_m2']>150) & (prop['price_usd_per_m2']<8000)]
len(prop)

70655

## Ordenamos el set de datos

In [6]:
p = prop[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','latlon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]
p.dropna(inplace = True)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
len(p)

70627

In [8]:
p['lat'] = p.apply(lambda row: row[4][0], axis=1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
p['lon'] = p.apply(lambda row: row[4][1], axis=1)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [10]:
p = p[['property_type','place_name','surface_total_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte']]

### Se le asigna un valor numerico al tipo de propiedad

In [11]:
def asignarPT( p ):
    if p == 'apartment':
        return 0
    if p == 'house':
        return 1
    if p == 'store':
        return 2
    return 3

In [12]:
p['property_type'] = p.apply(lambda x: asignarPT(x[0]), axis = 1)

### Se le asigna un valor a los barrios segun el analisis de grupos hecho en el tp1

In [13]:
def asignarPV(name, dic):
    return dic[name]   


In [14]:
hoodGroups = p[["place_name", "price_usd_per_m2"]]
hoodGroups = hoodGroups.groupby("place_name").agg([np.mean]).reset_index()
hoodGroups = hoodGroups.sort_values(by=("price_usd_per_m2", "mean"), ascending=False)

In [15]:
HGdics = {}
precio_ant = 0
val_ant = 0
for row in hoodGroups.iterrows():
    name = row[1][0]
    price = row[1][1]
    val = val_ant + (precio_ant/price)*10
    HGdics[name] = val
    precio_ant = price
    val_ant = val

In [16]:
p["place_value"] = p.apply(lambda x: asignarPV(x[1], HGdics), axis=1)

In [17]:
p = p[['property_type','surface_total_in_m2',\
          'price_usd_per_m2','lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos',
         'Cant_ParadasTransporte', 'place_value']]

# Random Forest Regresivo

In [18]:
#split del dataset para entrenamiento y pruebas, 80% y 20% respectivamente
Xtrn, Xtest, Ytrn, Ytest = train_test_split(p[['property_type','surface_total_in_m2',
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos','Cant_ParadasTransporte',
        'place_value']],p[['price_usd_per_m2']],test_size=0.2)

In [19]:
len(Xtrn), len(Xtest), len(Ytrn), len(Ytest)

(56501, 14126, 56501, 14126)

In [20]:
Xtest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14126 entries, 2290 to 3874
Data columns (total 8 columns):
property_type                14126 non-null int64
surface_total_in_m2          14126 non-null float64
lat                          14126 non-null float64
lon                          14126 non-null float64
Cant_ColeYUniv               14126 non-null int64
Cant_LocalesGastronomicos    14126 non-null int64
Cant_ParadasTransporte       14126 non-null int64
place_value                  14126 non-null float64
dtypes: float64(4), int64(4)
memory usage: 993.2 KB


##### Buscamos los n estimadores a mano(mientras mas grandes mejor)

In [20]:
models = []
tmp = []

models.append((RandomForestRegressor(n_estimators=10, max_features='sqrt'), 10))
models.append((RandomForestRegressor(n_estimators=100, max_features='sqrt'), 100))
models.append((RandomForestRegressor(n_estimators=1000, max_features='sqrt'), 1000))
#models.append((RandomForestRegressor(n_estimators=10000, max_features='sqrt'), 10000))
#models.append((RandomForestRegressor(n_estimators=100000, max_features='sqrt'), 100000))
#models.append((RandomForestRegressor(n_estimators=1000000, max_features='sqrt'), 1000000))    

In [21]:
#for i in xrange(0, len(models)):
#    modelN = models[i]
model = RandomForestRegressor(n_estimators=10, max_features='auto', min_samples_leaf=1, min_samples_split=4, bootstrap=True)
n = 10

#fit model on training dataset
model.fit(Xtrn, Ytrn['price_usd_per_m2'])
# predict prices for test dataset and calculate RMSE
print( ('n', n), ('RMSE_Price', mean_squared_error(Ytest['price_usd_per_m2'], model.predict(Xtest))) )


(('n', 10), ('RMSE_Price', 326798.26227937604))


In [22]:
print( ('n', n), ('RMSE_Price', mean_squared_error(Ytrn['price_usd_per_m2'], model.predict(Xtrn))) )

(('n', 10), ('RMSE_Price', 109427.44204369487))


In [29]:
len(Ytrn)

53554

In [23]:
#for i in xrange(3, len(models)):
#    modelN = models[i]
#    model = modelN[0]
#    n = modelN[1]
#
#    # fit model on training dataset
#    model.fit(Xtrn, Ytrn['price_usd_per_m2'])
#    # predict prices for test dataset and calculate RMSE
#    tmp.append(( ('n', n), ('RMSE_Price', mean_squared_error(Ytest['price_usd_per_m2'], model.predict(Xtest))) ))

#tmp

In [24]:
#[(('n', 10), ('RMSE_Price', 287547.43548892933)),
# (('n', 100), ('RMSE_Price', 265949.01332351164)),
# (('n', 1000), ('RMSE_Price', 262988.0550813392))]

#con 10000 se colgo la compu, y no hay mucha diferencia entre 100 y 1000
#entonces por temas de velocidad vamos a probar con n=1000 el resto de los
#hiperparametros

## GridSearch para busqueda de los mejores parametros

In [25]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [72]:
def Grid_Search_RF(Xtrn, Ytrn):
    estimator = RandomForestRegressor()

    param_grid = { 
            "n_estimators"      : [100],
            "max_features"      : ["auto"],
            "min_samples_split" : [2,3,4,5,6,7,8,9,10] ,
            "min_samples_leaf" : [1,2,3,4,5,6,7,8,9,10] ,
            "bootstrap": [True]
            #"oob_score" : [True, False],
            }

    grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5)

    grid.fit(Xtrn, Ytrn)

    return grid.best_score_ , grid.best_params_

In [30]:
results = []

In [58]:
for i in range(1):
    print "Loop: " , i
    print "--------------"
    
    #cross validation
    Xtrn, Xtest, Ytrn, Ytest = train_test_split(p[['property_type','surface_total_in_m2',
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos','Cant_ParadasTransporte',
        'place_value']],p[['price_usd_per_m2']],test_size=0.2)
    
    #grid search
    best_score, best_params = Grid_Search_RF(Xtrn, Ytrn['price_usd_per_m2'])
    
    #prueba de los mejores resultados
    estimator = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
    estimator.fit(Xtrn,Ytrn['price_usd_per_m2'])
    
    #prediccion 
    pred = mean_squared_error(Ytest['price_usd_per_m2'], estimator.predict(Xtest))
    print ('RMSE_Price', pred)
    
    #mejores resultados
    print "Best Score:" ,best_score
    print "Best params:",best_params
    results.append((pred, best_score, best_params))

Loop:  0
--------------
('RMSE_Price', 306614.58589863975)
Best Score: 0.764753636355
Best params: {'max_features': 'auto', 'min_samples_split': 10, 'bootstrap': True, 'n_estimators': 5000, 'min_samples_leaf': 10}


In [73]:
for i in range(100):
    print "Loop: " , i
    print "--------------"
    
    #cross validation
    Xtrn, Xtest, Ytrn, Ytest = train_test_split(p[['property_type','surface_total_in_m2',
        'lat','lon','Cant_ColeYUniv','Cant_LocalesGastronomicos','Cant_ParadasTransporte',
        'place_value']],p[['price_usd_per_m2']],test_size=0.2)
    
    #grid search
    best_score, best_params = Grid_Search_RF(Xtrn, Ytrn['price_usd_per_m2'])
    
    #prueba de los mejores resultados
    estimator = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
    estimator.fit(Xtrn,Ytrn['price_usd_per_m2'])
    
    #prediccion 
    pred = mean_squared_error(Ytest['price_usd_per_m2'], estimator.predict(Xtest))
    print ('RMSE_Price', pred)
    
    #mejores resultados
    print "Best Score:" ,best_score
    print "Best params:",best_params
    results.append((pred, best_score, best_params))

Loop:  0
--------------
('RMSE_Price', 271965.53344647208)
Best Score: 0.788472639604
Best params: {'max_features': 'auto', 'min_samples_split': 4, 'bootstrap': True, 'n_estimators': 100, 'min_samples_leaf': 1}
Loop:  1
--------------


KeyboardInterrupt: 

In [60]:
rp = results

In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [61]:
a = sorted(rp)
a

[(297529.95409271808,
  0.75584785250852127,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 10,
   'min_samples_split': 10,
   'n_estimators': 10}),
 (306614.58589863975,
  0.76475363635494176,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 10,
   'min_samples_split': 10,
   'n_estimators': 5000}),
 (563481.10336488229,
  679837.28809608694,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 500,
   'min_samples_split': 10000,
   'n_estimators': 100}),
 (587726.2124809901,
  666831.21776567202,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 500,
   'min_samples_split': 10000,
   'n_estimators': 100}),
 (1251751.1346537969,
  1319259.6697528884,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 6000,
   'min_samples_split': 100000,
   'n_estimators': 100}),
 (1256131.6429664805,
  1318162.0355778511,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf':

In [46]:
    param_grid = { 
            "n_estimators"      : [10],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [10, 50, 100, 1000, 5000, 10000, 50000, 100000, 150000] ,
            "min_samples_leaf" : [10, 50, 100, 1000, 5000, 10000, 50000, 100000, 150000],
            "bootstrap": [True]
            #"oob_score" : [True, False],
            }

In [22]:
pickle.dump(results, open("resultadosRF.p", "wb"))

NameError: name 'results' is not defined

In [23]:
resultsCargados = pickle.load(open("resultadosRF.p", "rb"))

In [24]:
resultsCargados

[(563481.10336488229,
  679837.28809608694,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 500,
   'min_samples_split': 10000,
   'n_estimators': 100}),
 (587726.2124809901,
  666831.21776567202,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 500,
   'min_samples_split': 10000,
   'n_estimators': 100}),
 (1251751.1346537969,
  1319259.6697528884,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 6000,
   'min_samples_split': 100000,
   'n_estimators': 100}),
 (1256131.6429664805,
  1318162.0355778511,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 500,
   'min_samples_split': 100000,
   'n_estimators': 100}),
 (1258993.0220926753,
  1317473.8641532564,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_samples_leaf': 1000,
   'min_samples_split': 100000,
   'n_estimators': 100}),
 (1259332.0448981086,
  1317373.2282638489,
  {'bootstrap': True,
   'max_features': 'auto',
   'min_sampl

In [33]:
Ytrn.head()

,price_usd_per_m2
48020,2857.142857
44462,2105.263158
57756,3076.923077
10171,800.000000
12186,1479.985833
